## Data Source
**MEV-Blocks API**

### Extraction Details
- **Flashbots Block Data:** Includes miner reward and exogenous variables such as gas price, gas used, coinbase transfers, etc.
- **Flashbots Transaction Data:** Focuses on transactions within Flashbots bundles.

In [1]:
import requests as r
import pandas as pd
import numpy as np

In [9]:
blocks_df = pd.DataFrame(columns=['block_number', 'miner_reward', 'fee_recipient_eth_diff', 'miner', 'fee_recipient', 'coinbase_transfers', 'eth_sent_to_fee_recipient', 'gas_used', 'gas_price', 'effective_priority_fee'])
transactions_df = pd.DataFrame(columns=['transaction_hash', 'tx_index', 'bundle_type', 'bundle_index', 'block_number', 'eoa_address', 'to_address', 'gas_used', 'gas_price', 'coinbase_transfer', 'eth_sent_to_fee_recipient', 'total_miner_reward', 'fee_recipient_eth_diff'])

blocks = r.get('https://blocks.flashbots.net/v1/blocks?limit=100').json()

for block in blocks['blocks']:
    transactions = block['transactions']
    del block['transactions']
    blocks_df = pd.concat([blocks_df, pd.DataFrame([block])], ignore_index=True)
    
    for transaction in transactions:
        transactions_df = pd.concat([transactions_df, pd.DataFrame([transaction])], ignore_index=True)

# since there is a max limit of 100 blocks per request, we loop to get earlier blocks
for i in range (0, 49):
    earliest_block = blocks_df['block_number'].iloc[-1]

    blocks = r.get('https://blocks.flashbots.net/v1/blocks?limit=100?before=' + str(earliest_block)).json()

    for block in blocks['blocks']:
        transactions = block['transactions']
        del block['transactions']
        blocks_df = pd.concat([blocks_df, pd.DataFrame([block])], ignore_index=True)
        
        for transaction in transactions:    
            transactions_df = pd.concat([transactions_df, pd.DataFrame([transaction])], ignore_index=True)

blocks_df
transactions_df

KeyboardInterrupt: 